## **Objectives**

---

Create a tool for sorting through a wishlist / deck of cards into 3 potential output list with proper formatting

- Output lists
    - [cardkingdom deck builder](https://www.cardkingdom.com/builder)
    - [mpcfill.com](https://mpcfill.com/)
    - list of cards that are already owned (cross referencing _collection_db)


<p style='page-break-after:always;'></p>

<br><br>

## **Inputs**

---

In [1]:
# input_csv_nm = 'mdfcs test list.csv'
# [ ] update this input to take in the standardized 'wish_list.csv' file
# input_csv_nm = 'wish_list.csv'
input_csv_nm = 'wish_list_glue_out'

price_limit_input = 5.00
# input_csv_name = 'pako value wish list'
input_csv_dir = 'C:\\git\mtg-proj\\wish_lists\\wish_list_glue_output\\'
input_csv_path = input_csv_dir+'\\'+input_csv_nm+'.csv'

### **Visualizing ETL pipeline (example)**

In [2]:
# # graphviz data pipeline example...
# import graphviz

# # Name of file and comment as arguments
# graphviz_file_name  = 'MTG Wishlist to Order sort tool'
# graphviz_comment    = 'MTG Wishlist to Order sort tool'
# pipeline_graph = graphviz.Digraph(graphviz_file_name, comment = graphviz_comment)

# # Definning alias and display name of each node
# pipeline_graph.node('0', 'Wish list input')
# pipeline_graph.node('1', 'Sorting tool')
# pipeline_graph.node('2', '')
# pipeline_graph.node('3', '')
# pipeline_graph.node('4', '')
# pipeline_graph.node('5', '')
# pipeline_graph.node('6', '')
# pipeline_graph.node('7', 'Alreay Owned List')
# pipeline_graph.node('8', 'MPCFill Order List')
# pipeline_graph.node('9', 'CardKingdom Order List')

# # Defining connections between nodes
# pipeline_graph.edges([
#         '01', 
#         # '17', '18', '19'
#         ])
# pipeline_graph.edge('1', '7', constraint='false')
# pipeline_graph.edge('1', '8', constraint='false')
# pipeline_graph.edge('1', '9', constraint='false')

# # save file to path
# #pipeline_graph.render(directory='graphviz-output')

# # printrender of graph
# pipeline_graph

In [3]:
#stop

<br><br><br><br>

### example scrython.cards. methods

---

- i.e.: 
```
card = scrython.cards.Id(id="5386a61c-4928-4bd1-babe-5b089ab8b2d9") 
card.name()       
```

- the above returns card name based on scryfall_id input

<br><br>

- `id()`
- `tcgplayer_id()`
- `name()`
- `rarity()`
- `set_code()`
- `set_name()`
- `colors()`
  - all colors in the mana cost
- `color_identity()`
  - all colors found on the card
- `cmc()`
  - converted mana cost
- `type_line()`
  - full type line
- `edhrec_rank()`
  - edhrec.com ranking
- `prices('usd')`
  - prices updated once every 24h

<br><br><br><br>

[scrython docs - cards.Id methods](https://github.com/NandaScott/Scrython/blob/main/docs/scrython.cards/Id.md)

<br><br><br>

## **Dependent Imports / Setup Environment**

---

In [4]:
import pandas as pd
from fp_data_toolbox import eda, fcal, notifier

### **Variable Setup**

In [5]:
###

<br><br><br><br>

## **Data Ingestion / Preprocessing**

---

### wish_list import

In [6]:
wish_list_input_df = pd.read_csv(input_csv_path)    

In [7]:
wish_list_stg = wish_list_input_df.rename(columns={"count": "count_needed", "name": "name"})

In [8]:
### objective: split MDFC names into front and back fields

### TODO [ ] adapt this for the possibility that MDFCs are split with '//' instead to '/'

    # there's a better way to do this
    # for loop
    # See below example
    # ====================================
        # nm_list = glued_data_df['name'].to_list()
        # upd_nm_list = []
        # nm_str_find_list = []

        # substr = "/ "
        # inserttxt = "/"
    # ====================================
        # for nm_str in nm_list:
        #     if nm_str.find(' / ') != -1:
        #         nm_str = (inserttxt+substr).join(nm_str.split(substr))
        #     if nm_str.find('//') == -1:
        #         upd_nm_list.append(nm_str)
        #         continue
        #     upd_nm_list.append(nm_str) # append list

        # glued_data_df['name'] = upd_nm_list
    # ====================================


nm_list = wish_list_stg['name'].to_list()
front_list = []
back_list = []


for nm_str in nm_list:
    front_nm_str = nm_str
    back_nm_str = nm_str
    if nm_str.find(' // ') != -1:
        front_nm_str = nm_str.split(' // ')[0]
        front_list.append(front_nm_str)
        back_nm_str = nm_str.split(' // ')[1]
        back_list.append(back_nm_str)
    if nm_str.find(' / ') != -1:
        front_nm_str = nm_str.split(' / ')[0]
        front_list.append(front_nm_str)
        back_nm_str = nm_str.split(' / ')[1]
        back_list.append(back_nm_str)
    if nm_str.find(' // ') == -1 and nm_str.find(' / ') == -1:
        front_list.append(nm_str)
        back_list.append('')


# splitting name fields into front / back fields for MPC input list formatting
# splitting on single / for input format from moxfield

# new data frame with split value columns
# split_df = wish_list_stg["name"].str.split("/", n = 1, expand = True)

# making separate first name column from new data frame
wish_list_stg["front"]= front_list
wish_list_stg["back"]= back_list
# making separate last name column from new data frame
# wish_list_stg["back"]= split_df[1]

#### reindex

In [9]:
wish_list_stg = wish_list_stg.reindex([
    'count_needed',
    'name',
    'front',
    'back',
], axis=1)

---

In [11]:
### setup lists for scryfall querying
scryfall_name_list = wish_list_stg['name'].tolist() # convert query key into list

scryfall_prices = [] # setup list for appending
index = len(scryfall_name_list) # create index for progress bar

In [12]:
### query scryfall for prices
import nest_asyncio
import scrython

nest_asyncio.apply() # neede to manage connection
i = 0 # setup variable for looping progress bar
for sf_nm in scryfall_name_list:
    card = scrython.cards.Named(exact=sf_nm) # pass in exact card names
    card_price_flt = card.prices('usd') # return prices
    scryfall_prices.append(card_price_flt) # append list
    i = i + 1 # iterate on integer
    progress_stat = str(round((i / index),2)) + ' / 1.0' # calc for progress
    print(progress_stat, end='\r') # print progress

wish_list_stg['price_est_usd'] = scryfall_prices

In [13]:
wish_list_stg = wish_list_stg.reindex([
    'name',
    'front',
    'back',
    'count_needed',
    'price_est_usd'
], axis=1)

### collection_db import

In [14]:
### import collection_db
collection_db_nm = 'collection_db.csv'
collection_db_dir = 'C:\\git\\mtg-proj\\collection_db\\'
collection_db_path = collection_db_dir+collection_db_nm
collection_db_df = pd.read_csv(collection_db_path)

In [15]:
### [ ] add some way to filter out decks that we consider "final" / do not want to canibalize from
    ### Potentially need to add a .csv as reference table of which card containers are off-limits for canibalization
collection_db_df = collection_db_df.reindex([
    'scryfall_id',
    'name',
    'front',
    'back',
    'count_owned',
    'price_est_usd'
], axis=1)

<br><br><br><br>

## **Data Cleaning**

---

In [16]:
wish_list_stg['count_needed']=[float(str(i).replace("None", "0")) for i in wish_list_stg['count_needed']]
wish_list_stg['count_needed']=wish_list_stg['count_needed'].fillna(0)
wish_list_stg['count_needed'] = wish_list_stg['count_needed'].astype(int)

In [17]:
collection_db_df['count_owned']=collection_db_df['count_owned'].fillna(0)
collection_db_df['price_est_usd']=collection_db_df['price_est_usd'].fillna(0)

collection_db_df['count_owned'] = collection_db_df['count_owned'].astype(int)
collection_db_df['price_est_usd'] = collection_db_df['price_est_usd'].astype(float)

<br><br><br><br>

## **Processing**

---

In [18]:
wish_list_fnl = wish_list_stg.groupby(['front']).agg({'count_needed': ['sum'],'price_est_usd': ['min']}) # aggregating on 'front'
wish_list_fnl.columns = ['count_needed','price_est_usd'] # flattening and renaming columns
wish_list_fnl = wish_list_fnl.reset_index() # resetting index

In [19]:
# aggregating collection db by 'front'
collection_db_df = collection_db_df.groupby(['front']).sum()

collection_db_df = collection_db_df.groupby(['front']).agg({'count_owned': ['sum']}) # aggregating on 'front'
collection_db_df.columns = ['count_owned'] # flattening and renaming columns
collection_db_df = collection_db_df.reset_index() # resetting index

### **Join collection_db and wish_list**

In [20]:
# join colleciton_db on to wish list
wish_list_joined_df=wish_list_stg.merge(collection_db_df, on='front', how='left')

In [21]:
# Sorting module here
# [ ] separate list of already owned cards into df

# data cleaning after merge
wish_list_joined_df['count_needed']=[float(str(i).replace("None", "0")) for i in wish_list_joined_df['count_needed']]
wish_list_joined_df['count_owned']=[float(str(i).replace("None", "0")) for i in wish_list_joined_df['count_owned']]
wish_list_joined_df['price_est_usd']=[float(str(i).replace("None", "0")) for i in wish_list_joined_df['price_est_usd']]

wish_list_joined_df['count_needed']=wish_list_joined_df['count_needed'].fillna(0)
wish_list_joined_df['count_owned']=wish_list_joined_df['count_owned'].fillna(0)
wish_list_joined_df['price_est_usd']=wish_list_joined_df['price_est_usd'].fillna(0)

wish_list_joined_df['count_needed'] = wish_list_joined_df['count_needed'].astype(int)
wish_list_joined_df['count_owned'] = wish_list_joined_df['count_owned'].astype(int)
wish_list_joined_df['price_est_usd']=wish_list_joined_df['price_est_usd'].astype(float)

In [22]:
mask = ((wish_list_joined_df['count_needed']<=wish_list_joined_df['count_owned']) | (wish_list_joined_df['count_owned']!=0))
already_owned_cards_df=wish_list_joined_df[mask]

In [23]:
non_owned_cards_df = wish_list_joined_df.query('count_needed > count_owned')
# non_owned_cards_df

## Formatting for outputs

In [24]:
mask = (non_owned_cards_df['price_est_usd']<=price_limit_input)
mkt_order_df = non_owned_cards_df[mask]

mask = (non_owned_cards_df['price_est_usd']>price_limit_input)
mpc_order_df = non_owned_cards_df[mask]

### Already owned

---

In [25]:
# [ ] finish this formatting
already_owned_cards_df=already_owned_cards_df
# already_owned_cards_df

### CK Order

---

In [26]:
# [ ] finish this formatting
ck_order_df = mkt_order_df.drop([
    'front',
    'back',
    'count_owned',
    'price_est_usd'
    ], axis=1)

ck_order_df = ck_order_df.reindex([
    'count_needed',
    'name'
    ], axis=1)

### TCGPlayer Order

---

In [27]:
# not needed for now
# tcg_order_df=mkt_order_df
# tcg_order_df = mkt_order_df.drop(columns=[
#     'front', 
#     'back',
#     'count_owned',
#     # 'count_needed',
#     'price_est_usd'
#     ])
# tcg_order_df = tcg_order_df.reindex([
#     'count_needed',
#     'name'
#     ])

In [28]:
# tcg_order_df

### MPC Order

---

In [29]:
# [ ] finish this formatting
mpc_order_df_fnl = mpc_order_df.drop([
    'front',
    'back',
    'count_owned',
    'price_est_usd'
    ], axis=1)

mpc_order_df_fnl = mpc_order_df_fnl.reindex([
    'count_needed',
    'name'
    ], axis=1)

In [30]:
#stop

<br><br><br><br>

## **Output**

---

- Already owned output

- CK Order output

- TCGPlayer Order output

- MPCFill Order output

- [x] notes on output formatting
    - formatting of data must match input platform for each
    - create new folder for each wish_list input
        - within that folder, all outputs



In [31]:
# [x] output dfs to .csv files in correct format

In [32]:
# [x] create folder based on wish_list file name input
output_order_name = input_csv_nm

from datetime import date
curr_dt = str(date.today())

output_order_folder_path = 'C:\\git\\mtg-proj\\order_sort_tool\\output'
output_order_folder_path_full = output_order_folder_path+'\\'+output_order_name+' - '+curr_dt
output_order_folder_path_full

'C:\\git\\mtg-proj\\order_sort_tool\\output\\wish_list_glue_out - 2022-08-26'

In [33]:
import os
# Create target directory & all intermediate directories if don't exists
if not os.path.exists(output_order_folder_path_full):
    os.makedirs(output_order_folder_path_full)
    print("Directory " , output_order_folder_path_full ,  " Created ")
else:    
    print("Directory " , output_order_folder_path_full ,  " already exists")

Directory  C:\git\mtg-proj\order_sort_tool\output\wish_list_glue_out - 2022-08-26  already exists


In [34]:
already_owned_csv_out_path = output_order_folder_path_full+'\\already_owned_cards.csv'

ck_order_csv_out_path = output_order_folder_path_full+'\\ck_order.csv'

# tcg_order_csv_out_path = ''+'tcg_order.csv'

mpc_order_csv_out_path = output_order_folder_path_full+'\\mpc_order.csv'

In [35]:
# [ ] execute output .csv files
already_owned_cards_df.to_csv(already_owned_csv_out_path,index=False)
ck_order_df.to_csv(ck_order_csv_out_path,index=False)
# tcg_order_df.to_csv(tcg_order_csv_out_path,index=False)
mpc_order_df.to_csv(mpc_order_csv_out_path,index=False)

---

In [ ]:
# stop